### Deliverable 2
* ping alternate citibikes route to save station information

In [1]:
import pandas as pd
import requests
import time

In [2]:
df = pd.read_csv('Resources/citibike_networks.csv')
df

,id,name,city,country,company,lat,lon
0,velobike-moscow,Velobike,Moscow,RU,ЗАО «СитиБайк»,55.750000,37.616667
1,bycyklen,Bycyklen,Copenhagen,DK,Gobike A/S,55.673582,12.564984
2,nu-connect,Nu-Connect,Utrecht,NL,Gobike A/S,52.117000,5.067000
3,baerum-bysykkel,Bysykkel,Bærum,NO,Urban Infrastructure Partner,59.894550,10.546343
4,bysykkelen,Bysykkelen,Stavanger,NO,Gobike A/S,58.969975,5.733107
...,...,...,...,...,...,...,...
656,semo,semo,Communauté d'agglomération Seine-Eure,FR,Ecovelo,49.220000,1.170000
657,cyclovis,Cyclovis,Soissons,FR,Ecovelo,49.381700,3.323600
658,chantrerie-captainbike,Chantrerie VLS (Captain Bike),"La Chantrerie, Nantes",FR,Ecovelo,47.286820,-1.521092
659,saclay-captainbike,Saclay VLS (Captain Bike),Paris-Saclay,FR,Ecovelo,48.709808,2.211366


In [5]:
# iterate through rows
for row in df.head(2).iterrows():
    
    print(type(row))
    
    # for each row, do something
#     print(type(x)) # iterrows returns a tuple = (row_number, data)
#     print(type(x[1])) # data returned as Series
#     print(x[1].name) # name attr is row number
#     print(x[1])
    
    
    print('-'*30)

<class 'tuple'>
------------------------------
<class 'tuple'>
------------------------------


In [8]:
url = 'http://api.citybik.es/v2/networks'

In [9]:
# calling a single iterator variable makes iterrows cumbersome
for x in df.head(2).iterrows():
    print(x[1])
    
    # get city
    city = x[1]['city']
    
    # make request
    req = requests.get(f'{url}/{city}') # we will receive 404
    print(req)
    print('-'*30)

id         velobike-moscow
name              Velobike
city                Moscow
country                 RU
company     ЗАО «СитиБайк»
lat                  55.75
lon                37.6167
Name: 0, dtype: object
<Response [404]>
------------------------------
id           bycyklen
name         Bycyklen
city       Copenhagen
country            DK
company    Gobike A/S
lat           55.6736
lon            12.565
Name: 1, dtype: object
<Response [404]>
------------------------------


#### When working with APIs, it can help to work directly with a handful of responses before iterating through all. You may even want to visit the route in a browser to inspect more closely.

In [10]:
my_id = 'velobike-moscow'
url = f'{url}/{my_id}'
url

'http://api.citybik.es/v2/networks/velobike-moscow'

In [15]:
# try single
req = requests.get(url)
resp = req.json()

# 1. check top level keys
resp.keys()

# 2. check keys for 'network' dict
resp['network']

# 3. check 'stations' value -- this is the array we need
stations_list = resp['network']['stations']

In [87]:
stations = []

for x in stations_list[:3]:
    
    # strip out values
    print(x.keys())
    stn_id = x['id']
    name = x['name']
    lat = x['latitude']
    lon = x['longitude']
    free_bikes = x['free_bikes']
    
    # set dict
    data = {
        'id':stn_id,
        'name':name,
        'lat':lat,
        'lon':lon,
        'free_bikes':free_bikes,
    }
    
    stations.append(data)

dict_keys(['empty_slots', 'extra', 'free_bikes', 'id', 'latitude', 'longitude', 'name', 'timestamp'])
dict_keys(['empty_slots', 'extra', 'free_bikes', 'id', 'latitude', 'longitude', 'name', 'timestamp'])
dict_keys(['empty_slots', 'extra', 'free_bikes', 'id', 'latitude', 'longitude', 'name', 'timestamp'])


In [88]:
stations

[{'id': 'c8a5761ea95e927808b2f37fd65a00f6',
  'name': 'ст. м. Кропоткинская (выход к Гоголевскому бульвару)',
  'lat': 55.7449006,
  'lon': 37.6020183,
  'free_bikes': 0},
 {'id': 'ac022f0470de9722cc62e06ade73b0a6',
  'name': 'ул. Профсоюзная, д.109, Вс.1 (ст. м. Коньково, выход №10)',
  'lat': 55.6337498,
  'lon': 37.5205529,
  'free_bikes': 0},
 {'id': '17ffd07cf6b96ecd32b4980353290399',
  'name': 'ул. Новолесная, д.1/49',
  'lat': 55.7812396,
  'lon': 37.59239,
  'free_bikes': 0}]

#### Now, try in loop

In [94]:
# empty array to store our station data
station_data = []
url = 'http://api.citybik.es/v2/networks'

# for index, row in df.head(2).iterrows(): # .head() is a handy method of iterating through just a few rows

for index, row in df.iterrows():
    print(index)
    
    # get id, city, country, name
    ntwk_id = row['id']
    ntwk_name = row['name']
    ntwk_city = row['city']
    ntwk_country = row['country']
    ntwk_company= row['company']
    
    # make request and confirm
    req = requests.get(f'{url}/{ntwk_id}')
    # print(req)
    
    # save json
    resp = req.json()
    
    # save stations array
    stations = resp['network']['stations']
    
    for station in stations:
        
        # try block for keys
        try:
            stn_id = station['id']
            stn_name = station['name']
            lat = station['latitude']
            lon = station['longitude']
            free_bikes = station['free_bikes']

            # set dict
            data = {
                'id':stn_id,
                'station_name':stn_name,
                'city':ntwk_city,
                'country':ntwk_country,
                'company':ntwk_company,
                'lat':lat,
                'lon':lon,
                'free_bikes':free_bikes,
            }

            # append dict to array
            station_data.append(data)
            
        # except any non-existing keys and skip that station
        except KeyError as e:
            print(f'{index}. KeyError ', e)
            pass
    
    # separator
    print('-'*30)
    
    # wait
    time.sleep(1)

0
------------------------------
1
------------------------------
2
------------------------------
3
------------------------------
4
------------------------------
5
------------------------------
6
------------------------------
7
------------------------------
8
------------------------------
9
------------------------------
10
------------------------------
11
------------------------------
12
------------------------------
13
------------------------------
14
------------------------------
15
------------------------------
16
------------------------------
17
------------------------------
18
------------------------------
19
------------------------------
20
------------------------------
21
------------------------------
22
------------------------------
23
------------------------------
24
------------------------------
25
------------------------------
26
------------------------------
27
------------------------------
28
------------------------------
29
---------------------

In [95]:
df = pd.DataFrame(station_data)
df

,id,station_name,city,country,company,lat,lon,free_bikes
0,c8a5761ea95e927808b2f37fd65a00f6,ст. м. Кропоткинская (выход к Гоголевскому бул...,Moscow,RU,ЗАО «СитиБайк»,55.744901,37.602018,0.0
1,ac022f0470de9722cc62e06ade73b0a6,"ул. Профсоюзная, д.109, Вс.1 (ст. м. Коньково,...",Moscow,RU,ЗАО «СитиБайк»,55.633750,37.520553,0.0
2,17ffd07cf6b96ecd32b4980353290399,"ул. Новолесная, д.1/49",Moscow,RU,ЗАО «СитиБайк»,55.781240,37.592390,0.0
3,2a711f9eb3228be91abce39eca68e118,"ул. Селезнёвская, д.29, стр.1 (ст. м. Достоевс...",Moscow,RU,ЗАО «СитиБайк»,55.781785,37.614473,0.0
4,70df0b4cae5a2057e8e78cccb4189832,"ул. Новосущёвская, д.22, стр.8",Moscow,RU,ЗАО «СитиБайк»,55.789663,37.604464,0.0
...,...,...,...,...,...,...,...,...
28510,81ae9cb28f00a50851f7df128465fbbf,Camping Gaalgebierg,Esch-sur-alzette,LU,CIGL ESCH,49.485050,5.986330,4.0
28511,07c6a1570c1cec8fa187886c2a4a0b79,Rue Ribeschpont,Esch-sur-alzette,LU,CIGL ESCH,49.493013,6.081453,3.0
28512,23a705350415254a5c39cc6cd79d53da,Centre Commercial Cora,Esch-sur-alzette,LU,CIGL ESCH,49.521320,6.010340,4.0
28513,62ecdb728827fb23fbee92b1be16f972,Zolwereck,Esch-sur-alzette,LU,CIGL ESCH,49.522762,5.921783,3.0


In [96]:
df.to_csv('Resources/citibike_stations.csv', index=False)